In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("dataset//City_Types.csv")

In [3]:
df.head(3)

,Date,City,CO,NO2,SO2,O3,PM2.5,PM10,Type
0,2024-01-01 00:00:00+00:00,Moscow,208.0,15.9,13.2,44.0,8.6,9.4,Industrial
1,2024-01-01 01:00:00+00:00,Moscow,207.0,17.4,13.7,44.0,8.6,10.5,Industrial
2,2024-01-01 02:00:00+00:00,Moscow,217.0,19.0,15.5,43.0,10.4,12.9,Industrial


In [4]:
df.isnull().sum()

Date     0
City     0
CO       0
NO2      0
SO2      0
O3       0
PM2.5    0
PM10     0
Type     0
dtype: int64

In [5]:
df.describe()

,CO,NO2,SO2,O3,PM2.5,PM10
count,52704.000000,52704.000000,52704.000000,52704.000000,52704.000000,52704.000000
mean,508.030472,29.616492,22.387250,53.423668,32.931045,50.644980
std,692.274824,23.984409,34.285823,42.225556,43.833691,67.311502
min,0.000000,0.900000,0.000000,0.000000,0.300000,0.400000
25%,187.000000,11.000000,0.700000,26.000000,6.400000,9.400000
50%,268.000000,23.300000,10.500000,48.000000,14.800000,19.800000
75%,519.000000,42.200000,30.200000,69.000000,42.600000,68.350000
max,12876.000000,218.000000,497.800000,342.000000,459.100000,661.200000


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train,X_test, y_train,y_test = train_test_split(df.iloc[:,1:8], df.iloc[:,-1], test_size=0.2, random_state=0)

In [8]:
X_train

,City,CO,NO2,SO2,O3,PM2.5,PM10
21718,Beijing,688.0,63.3,46.2,11.0,59.6,86.4
7148,Moscow,202.0,21.1,11.3,40.0,10.3,11.7
7971,Moscow,536.0,48.8,117.5,0.0,70.4,86.0
31048,Zurich,136.0,5.1,0.3,93.0,4.2,6.4
25144,Beijing,2965.0,20.1,234.3,0.0,259.8,373.5
...,...,...,...,...,...,...,...
45891,Stockholm,165.0,2.4,0.2,72.0,2.7,6.6
52416,Stockholm,154.0,5.2,0.3,58.0,3.1,6.1
42613,Vancouver,556.0,28.4,8.2,25.0,11.9,18.5
43567,Vancouver,445.0,28.6,4.1,48.0,6.1,6.5


In [9]:
X_train.nunique()

City        6
CO       2966
NO2      1356
SO2      1850
O3        296
PM2.5    1358
PM10     1590
dtype: int64

In [14]:
y_train.value_counts()

Type
Industrial     21099
Residential    21064
Name: count, dtype: int64

***So far in the dataset we know that there are 2 unique values in type and 6 unique values in city***

***First we will perform scaling of the input dataset***

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [26]:
ct = ColumnTransformer(transformers=[
    ('tf1',StandardScaler(),['CO','NO2','SO2','O3','PM2.5','PM10']),
    ('tf2',OneHotEncoder(sparse_output=False,dtype=np.int32,drop='first'), ['City'])
], remainder='passthrough')

In [32]:
X_train_transformed_standard = ct.fit_transform(X_train)

In [33]:
X_test_transformed_standard = ct.transform(X_test)

In [34]:
from sklearn.preprocessing import LabelEncoder

In [39]:
le = LabelEncoder()
y_train_transformed = le.fit_transform(y_train)
y_train_transformed

array([0, 0, 0, ..., 1, 1, 0])

In [38]:
y_train

21718     Industrial
7148      Industrial
7971      Industrial
31048    Residential
25144     Industrial
            ...     
45891    Residential
52416    Residential
42613    Residential
43567    Residential
2732      Industrial
Name: Type, Length: 42163, dtype: object

In [40]:
y_test_transformed = le.transform(y_test)

In [41]:
from sklearn.linear_model import LogisticRegression

In [42]:
model = LogisticRegression()

In [43]:
model.fit(X_train_transformed_standard, y_train_transformed)

LogisticRegression()

In [45]:
y_pred = model.predict(X_test_transformed_standard)

In [46]:
from sklearn.metrics import accuracy_score

In [47]:
score = accuracy_score(y_test_transformed, y_pred)

In [48]:
score

1.0

***We got a score of 1(signs of overtrained)***

Using Decision Tree classifier

In [49]:
from sklearn.tree import DecisionTreeClassifier
model_DTC = DecisionTreeClassifier()

In [50]:
model_DTC.fit(X_train_transformed_standard, y_train_transformed)

DecisionTreeClassifier()

In [51]:
y_pred_dtc = model_DTC.predict(X_test_transformed_standard)

In [52]:
score_dtc = accuracy_score(y_test_transformed, y_pred_dtc)

In [53]:
score_dtc

0.9993359263826962

In [54]:
import matplotlib.pyplot as plt 

In [55]:
importances = model_DTC.feature_importances_

In [56]:
feature_names = ct.get_feature_names_out()

In [57]:
plt.figure(figsize=(10,6))
plt.barh(feature_names, importances)
plt.xlabel('Feature Importance')
plt.title('Decision Tree Feature Importances')
plt.show()

***This suggests that the SO2 values has a great impact on classifying the type of Air***